In [32]:
import nflreadpy as nfl
import pandas as pd
import duckdb
import polars as pl
pd.set_option('display.max_columns',30)

In [11]:
con = duckdb.connect("data/nfl.duckdb")

In [ ]:
end_year = 2026
years=range(2000,end_year)

In [ ]:
weekly = nfl.load_player_stats(range(2000,end_year))
weekly.write_parquet('data/weekly.parquet')
con.execute("CREATE OR REPLACE TABLE weekly AS SELECT * FROM weekly")

Downloading: 100%|██████████| 579k/579k [00:00<00:00, 18.1MB/s]


Downloading: 100%|██████████| 585k/585k [00:00<00:00, 117MB/s]


Downloading: 100%|██████████| 607k/607k [00:00<00:00, 152MB/s]


Downloading: 100%|██████████| 529k/529k [00:00<00:00, 106MB/s]


Downloading: 100%|██████████| 526k/526k [00:00<00:00, 175MB/s]


...

Downloading: 100%|██████████| 767k/767k [00:00<00:00, 153MB/s]


Downloading: 100%|██████████| 759k/759k [00:00<00:00, 94.9MB/s]


Downloading: 100%|██████████| 758k/758k [00:00<00:00, 94.8MB/s]


Downloading: 100%|██████████| 762k/762k [00:00<00:00, 152MB/s]


Downloading: 100%|██████████| 144k/144k [00:00<?, ?B/s]


In [7]:
pbp = nfl.load_pbp(years)

Downloading: 100%|██████████| 13.4M/13.4M [00:02<00:00, 6.63MB/s]


Downloading: 100%|██████████| 14.0M/14.0M [00:02<00:00, 6.45MB/s]


Downloading: 100%|██████████| 14.8M/14.8M [00:02<00:00, 5.92MB/s]


Downloading: 100%|██████████| 14.6M/14.6M [00:02<00:00, 6.14MB/s]


Downloading: 100%|██████████| 14.6M/14.6M [00:02<00:00, 6.12MB/s]


...

Downloading: 100%|██████████| 20.3M/20.3M [00:04<00:00, 4.47MB/s]


Downloading: 100%|██████████| 20.4M/20.4M [00:04<00:00, 4.31MB/s]


Downloading: 100%|██████████| 20.5M/20.5M [00:04<00:00, 4.33MB/s]


Downloading: 100%|██████████| 20.6M/20.6M [00:04<00:00, 4.40MB/s]


Downloading: 100%|██████████| 1.42M/1.42M [00:00<00:00, 59.3MB/s]


In [8]:
pbp.write_parquet('data/pbp.parquet')

In [15]:
#pbp.write_database('pbp', 'sqlite:///data/nfl.db', if_table_exists='replace')
con.execute("CREATE OR REPLACE TABLE pbp AS SELECT * FROM pbp")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
rosters = nfl.load_rosters(range(2002,end_year))
rosters

Downloading: 100%|██████████| 247k/247k [00:00<00:00, 123MB/s]


Downloading: 100%|██████████| 250k/250k [00:00<00:00, 83.5MB/s]


Downloading: 100%|██████████| 255k/255k [00:00<00:00, 255MB/s]


Downloading: 100%|██████████| 259k/259k [00:00<00:00, 129MB/s]


Downloading: 100%|██████████| 263k/263k [00:00<00:00, 132MB/s]


...

Downloading: 100%|██████████| 560k/560k [00:00<00:00, 70.0MB/s]


Downloading: 100%|██████████| 555k/555k [00:00<00:00, 79.2MB/s]


Downloading: 100%|██████████| 566k/566k [00:00<00:00, 56.6MB/s]


Downloading: 100%|██████████| 516k/516k [00:00<00:00, 64.5MB/s]


season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,weight,college,gsis_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number
i32,str,str,str,str,str,str,str,str,date,f64,i32,str,str,str,str,str,str,str,str,str,str,i32,str,str,i32,str,str,str,str,str,str,i32,i32,str,str
2002,"""CHI""","""RB""",null,"""27""","""ACT""","""Rabih Abdullah""","""Rabih""","""Abdullah""",1975-04-27,72.0,220,null,"""00-0000007""",null,null,null,null,null,null,null,null,4,"""https://static.www.nfl.com/ima…",null,17,"""REG""","""A01""","""Rabih""","""ABD675101""",null,"""32004142-4467-5101-8326-4b71f9…",1998,1998,null,null
2002,"""NYJ""","""CB""",null,"""29""","""ACT""","""Donnie Abraham""","""Nathaniel""","""Abraham""",1973-10-08,70.0,192,null,"""00-0000017""",null,null,null,null,null,null,null,null,6,"""https://static.www.nfl.com/ima…",null,19,"""DIV""","""A01""","""Donnie""","""ABR073258""",null,"""32004142-5207-3258-ebd3-814f82…",1996,1996,"""TB""","""71"""
2002,"""TEN""","""G""",null,"""67""","""ACT""","""Tom Ackerman""","""Thomas""","""Ackerman""",1972-09-06,75.0,292,null,"""00-0000032""",null,null,null,null,null,null,null,null,6,"""https://static.www.nfl.com/ima…",null,20,"""CON""","""A01""","""Tom""","""ACK425040""",null,"""32004143-4b42-5040-5e4c-ffc269…",1996,1996,"""NO""","""145"""
2002,"""DAL""","""T""",null,"""76""","""ACT""","""Flozell Adams""","""Flozell""","""Adams""",1975-05-18,79.0,357,null,"""00-0000045""",null,null,null,null,null,null,null,null,4,"""https://static.www.nfl.com/ima…",null,17,"""REG""","""A01""","""Flozell""","""ADA280969""",null,"""32004144-4128-0969-9390-de3ae3…",1998,1998,"""DAL""","""38"""
2002,"""OAK""","""DT""",null,"""95""","""ACT""","""Sam Adams""","""Sam""","""Adams""",1973-06-13,75.0,330,null,"""00-0000065""",null,null,null,null,null,null,null,null,8,"""https://static.www.nfl.com/ima…",null,21,"""SB""","""A01""","""Sam""","""ADA571656""",null,"""32004144-4157-1656-c9b5-9681a9…",1994,1994,"""SEA""","""8"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025,"""KC""","""LB""","""LB""","""49""","""CUT""","""Xander Mueller""","""Xander""","""Mueller""",null,null,232,null,"""00-0040780""",null,null,null,null,null,null,null,null,0,null,null,1,"""REG""","""W03""","""Xander""","""MUE622935""","""59264""","""32004d55-4562-2935-86f2-6931bc…",2025,2025,null,null
2025,"""IND""","""LB""","""LB""","""49""","""CUT""","""Joseph Vaughn""","""Joseph""","""Vaughn""",null,null,238,null,"""00-0040781""",null,null,null,null,null,null,null,null,0,null,null,1,"""REG""","""W03""","""Joseph""","""VAU443667""","""59277""","""32005641-5544-3667-4846-303a16…",2025,2025,null,null
2025,"""CLE""","""WR""","""WR""","""16""","""ACT""","""Isaiah Bond""","""Isaiah""","""Bond""",2004-03-15,71.0,180,"""Texas; Alabama""","""00-0040782""",null,null,null,null,null,null,null,null,0,"""https://static.www.nfl.com/ima…",null,2,"""REG""","""A01""","""Isaiah""","""BON168854""","""59278""","""3200424f-4e16-8854-5500-4fcd95…",2025,2025,null,null


In [18]:
#rosters.to_sql('rosters', conn, if_exists='replace', index=False)
con.execute("CREATE OR REPLACE TABLE rosters AS SELECT * FROM rosters")

In [25]:
ids = nfl.load_ff_playerids()
ids

Downloading: 2.52MB [00:00, 19.2MB/s]                  


mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
i64,str,i64,str,str,i64,str,i64,str,str,i64,str,str,i64,str,i64,i64,i64,str,str,str,str,str,str,str,f64,i64,i64,i64,str,str,i64,i64,str,i64
17030,"""3c76cab3-3df2-43dd-acaa-57e055…",24755,null,null,12522,null,4688380,null,null,3168422,null,null,16997,null,1730,41786,0,null,null,"""Cam Ward""","""cam ward""","""QB""","""TEN""","""2002-05-25""",23.3,2025,1,1,null,null,74,219,"""Miami (FL)""",2025
17031,"""270e09bc-8bf8-44b0-87ed-1fd014…",25968,null,null,12524,null,4432762,null,null,26710004,null,null,18479,null,1731,41930,0,null,null,"""Shedeur Sanders""","""shedeur sanders""","""QB""","""CLE""","""2002-02-07""",23.6,2025,5,6,null,null,74,212,"""Colorado""",2025
17032,"""53848cba-bcad-4cd0-bb35-0a1f6e…",23160,null,null,12508,null,4689114,null,null,26694019,null,null,18574,null,1732,41810,0,null,null,"""Jaxson Dart""","""jaxson dart""","""QB""","""NYG""","""2003-05-13""",22.3,2025,1,25,null,null,74,223,"""Ole Miss""",2025
17033,"""b0eb5b67-4ef2-462d-bfe7-e73bcc…",22884,null,null,12510,null,4432734,null,null,26694943,null,null,18477,null,1733,41882,0,null,null,"""Jalen Milroe""","""jalen milroe""","""QB""","""SEA""","""2002-12-13""",22.7,2025,3,28,null,null,74,216,"""Alabama""",2025
17034,"""d89f0384-21a6-44e4-8aa6-324cb2…",23680,null,null,12500,null,4889929,null,null,26746721,null,null,18505,null,1734,42037,0,null,null,"""Quinn Ewers""","""quinn ewers""","""QB""","""MIA""","""2003-05-13""",22.3,2025,7,15,null,null,74,209,"""Texas""",2025
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
972,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Doug Brien""","""doug brien""","""PK""","""NOS""",null,null,null,null,null,null,null,null,null,null,2025
978,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Jeremy Brigham""","""jeremy brigham""","""TE""","""OAK""",null,null,null,null,null,null,null,null,null,null,2025
984,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Vincent Brisby""","""vincent brisby""","""WR""","""FA""",null,null,null,null,null,null,null,null,null,null,2025


In [ ]:
#ids.to_sql('ids', conn, if_exists='replace', index=False)
con.execute("CREATE OR REPLACE TABLE ids AS SELECT * FROM ids")

In [36]:
snaps = nfl.load_snap_counts(range(2012,end_year))
team_map = {'OAK':'LV',
            'STL':'LA',
            'SD':'LAC'}
snaps = snaps.to_pandas()  # temporarily convert
snaps['team'] = snaps['team'].map(lambda x: team_map.get(x,x))
snaps = pl.from_pandas(snaps)
#snaps.to_sql('snaps', conn, if_exists='replace', index=False)
con.execute("CREATE OR REPLACE TABLE snaps AS SELECT * FROM snaps")

In [40]:
depth = nfl.load_depth_charts(range(2001,end_year)).unique()

team_map = {'OAK':'LV',
            'STL':'LA',
            'SD':'LAC'}

depth = depth.to_pandas()  # temporarily convert
depth['club_code'] = depth['club_code'].apply(lambda x: team_map.get(x,x))
depth = depth.sort_values(by=['season','club_code','week','depth_team'])
depth['dense_depth'] = depth.groupby(['club_code','season','week','position', 'game_type','formation']).cumcount()
depth['depth_position'] = depth['depth_position'].replace({'\n    ':None})
depth['depth_position'] = depth['depth_position'].fillna(depth['position'])
depth = depth.rename({'club_code':'team_name'},axis=1)

#depth.to_sql('depth', conn, if_exists='replace', index=False)
con.execute("CREATE OR REPLACE TABLE depth AS SELECT * FROM depth")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [41]:
schedule = nfl.load_schedules(years)
#schedule.to_sql('schedule', conn, if_exists='replace', index=False)
con.execute("CREATE OR REPLACE TABLE schedule AS SELECT * FROM schedule")
schedule

Downloading: 2.09MB [00:00, 25.5MB/s]                  


game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
str,i64,str,i64,str,str,str,str,i64,str,i64,str,i64,i64,i64,i64,i64,str,str,str,i64,str,i64,i64,str,str,f64,str,str,f64,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str
"""2000_01_SF_ATL""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""SF""",28,"""ATL""",36,"""Home""",8,64,0,2000090310,998,null,"""200009030atl""",null,200903001,null,7,7,null,null,6.5,null,null,46.5,null,null,1,"""dome""","""astroturf""",null,null,"""00-0005755""","""00-0002876""","""Jeff Garcia""","""Chris Chandler""","""Steve Mariucci""","""Dan Reeves""","""Walt Coleman""","""ATL00""","""Georgia Dome"""
"""2000_01_JAX_CLE""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""JAX""",27,"""CLE""",7,"""Home""",-20,34,0,2000090306,994,null,"""200009030cle""",null,200903005,null,7,7,null,null,-10.5,null,null,38.5,null,null,1,"""outdoors""","""grass""",78,6,"""00-0002110""","""00-0003535""","""Mark Brunell""","""Tim Couch""","""Tom Coughlin""","""Chris Palmer""","""Bernie Kukar""","""CLE00""","""Cleveland Browns Stadium"""
"""2000_01_IND_KC""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""IND""",27,"""KC""",14,"""Home""",-13,41,0,2000090305,993,null,"""200009030kan""",null,200903012,null,7,7,null,null,-3.5,null,null,44.0,null,null,0,"""outdoors""","""grass""",90,5,"""00-0010346""","""00-0006300""","""Peyton Manning""","""Elvis Grbac""","""Jim Mora""","""Gunther Cunningham""","""Larry Nemmers""","""KAN00""","""Arrowhead Stadium"""
"""2000_01_CHI_MIN""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""CHI""",27,"""MIN""",30,"""Home""",3,57,0,2000090303,991,null,"""200009030min""",null,200903016,null,7,7,null,null,4.5,null,null,46.5,null,null,1,"""dome""","""astroturf""",null,null,"""00-0011044""","""00-0003739""","""Cade McNown""","""Daunte Culpepper""","""Dick Jauron""","""Dennis Green""","""Ed Hochuli""","""MIN00""","""Hubert H. Humphrey Metrodome"""
"""2000_01_TB_NE""",2000,"""REG""",1,"""2000-09-03""","""Sunday""","""13:00""","""TB""",21,"""NE""",16,"""Home""",-5,37,0,2000090312,1000,null,"""200009030nwe""",null,200903017,null,7,7,null,null,-3.0,null,null,36.0,null,null,0,"""outdoors""","""grass""",71,5,"""00-0009266""","""00-0001361""","""Shaun King""","""Drew Bledsoe""","""Tony Dungy""","""Bill Belichick""","""Jeff Triplette""","""BOS99""","""Foxboro Stadium"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025_18_DAL_NYG""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""DAL""",null,"""NYG""",null,"""Home""",null,null,null,2025010411,null,null,"""202501040nyg""","""28684""",401772963,"""7001""",6,6,null,null,null,null,null,null,null,null,1,"""outdoors""","""fieldturf""",null,null,null,null,null,null,"""Brian Schottenheimer""","""Brian Daboll""",null,"""NYC01""","""MetLife Stadium"""
"""2025_18_WAS_PHI""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""WAS""",null,"""PHI""",null,"""Home""",null,null,null,2025010412,null,null,"""202501040phi""","""28685""",401772959,"""7002""",6,6,null,null,null,null,null,null,null,null,1,"""outdoors""","""grass""",null,null,null,null,null,null,"""Dan Quinn""","""Nick Sirianni""",null,"""PHI00""","""Lincoln Financial Field"""
"""2025_18_BAL_PIT""",2025,"""REG""",18,"""2026-01-04""","""Sunday""","""13:00""","""BAL""",null,"""PIT""",null,"""Home""",null,null,null,2025010413,null,null,"""202501040pit""","""28686""",401772960,"""7003""",10,10,null,null,null,null,null,null,null,null,1,"""outdoors""","""grass""",null,null,null,null,null,null,"""John Harbaugh""","

In [43]:
con.commit()
con.close()